## 7.1 Learning meaning
Bản chất của các từ và bí mật giữa chúng có tương quan với nhau. Mối tương quan có thể được hiểu bằng ít nhất 2 cách

1. Word order: 2 câu sau đây có ý nghĩa khác nhau:

`Con chó đuổi theo con mèo`

`Con mèo đuổi theo con chó`

2. Word proximity: (trạng thái từ): Câu này từ sáng rực rỡ ở cuối câu nói về thân tàu ở đầu câu.

`Thân tàu, mặc dù nằm trên biển, hàng triệu tấn hàng, sáng rực rỡ như mới`

## 7.2 Toolkit
Hai trong số các chương trình chuyên về kiến trúc mạng neural là Theano và Tensorflow. 

Facebook nỗ lực đưa ngôn ngữ Lua vào gọi là Torch; Python có API giống như vậy gọi là PyTorch. 

Lasagne (Theano) và Skflow (Tensorflow) là lựa chọn phổ biến, nhưng chúng tôi sẽ dùng Keras (https://keras.io) (thân thiện và linh hoạt). Keras có thể sử dụng Tensorflow hoặc Theano là backend, và mỗi cái đều có ưu và nhược điểm của nó, nhưng bạn sẽ sử dụng Tensorflow cho các ví dụ. Bạn cần package h5py để lưu mô hình đã train của bạn.

Mặc định, Keras sử dụng Tensorflow là backend.

## 7.3 Convolutional neural nét

## 7.4 Narrow windows indeed

Ừ, ừ, được rồi, hình ảnh. Nhưng chúng ta đang nói về ngôn ngữ ở đây, nhớ không? Hãy xem nào
một số từ ngữ để đào tạo trên. Hóa ra bạn có thể sử dụng các mạng thần kinh xoắn cho
chế biến tự nhiên ngôn ngữ bằng cách sử dụng các vector từ (còn gọi là embeddings từ),
mà bạn đã học về trong chương 6, thay vì giá trị pixel của hình ảnh, như đầu vào
vào mạng của bạn.

Bạn muốn chỉ tập trung vào các mối quan hệ của
mã thông báo trong một chiều không gian. Thay vì một
lọc hai chiều mà bạn sẽ Convolve
trên một đầu vào hai chiều (ảnh), bạn sẽ
bộ lọc onedimensional dây leo trên một đầu vào onedimensional, chẳng hạn như một câu

hình dạng bộ lọc của bạn cũng sẽ có một chiều
thay vì hai chiều như trong cửa sổ 1 x 3 cán thể hiện trong hình 7.7

<img src="./images/conv1d_word.png"/>
Nếu bạn tưởng tượng văn bản như một hình ảnh, “thứ hai” chiều là toàn bộ chiều dài của
vector từ, thường 100-D - 500-D, giống như một hình ảnh thực sự. Chúng tôi sẽ chỉ liên quan đến việc
các “width” của bộ lọc. Trong hình 7.7, bộ lọc là ba tokens rộng. Aha! Thông báo rằng
mỗi thẻ từ (hay muộn nhân vật token) là một “pixel” trong câu của bạn “hình ảnh


<img src="./images/conv1d_embed_word.png"/>

### 7.4.1 Thực thi trong Keras: Chuẩn bị dữ liệu
Chúng ta hãy xem về convolution trong Python với ví dụ phân lớp convolution neural network trong documentation Keras. Dữ liệu phim IMDB.

Mỗi điểm dữ liệu là đã gán nhãn 0 (tiêu cực) và 1 (tích cực). 


In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D

Using TensorFlow backend.


Đầu tiên, tải dữ liệu gốc của Stanford AI (https://ai.stanford.edu/%7eamaas/data/sentiment/). Đây là dataset biên soạn năm 2011 paper `Learning Word Vectors for Sentiment Analysis`. Khi bạn đã tải dataset về, giải nén nó ra một thư mục và tìm hiểu nó. Bạn sẽ chỉ sử dụng data trong thư mục train để train model



In [82]:
import glob
import os
from random import shuffle

def pre_process_data(filepath):
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')
    
    pos_label = 1
    neg_label = 0
    dataset = []
    
    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        print(filename)
        try:
            f = open(filename, 'r', encoding='utf8')
            dataset.append((pos_label, f.read()))
            #with open(filename, 'r', encoding='utf8', errors = 'ignore') as f:
            #    dataset.append((pos_label, f.read()))
        except:
            pass
            
    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((neg_label, f.read()))
    
    shuffle(dataset)
    
    return dataset

dataset = pre_process_data('./data/aclimdb/train/')

./data/aclimdb/train/pos\0_9.txt
./data/aclimdb/train/pos\10000_8.txt
./data/aclimdb/train/pos\10001_10.txt
./data/aclimdb/train/pos\10002_7.txt
./data/aclimdb/train/pos\10003_8.txt
./data/aclimdb/train/pos\10004_8.txt
./data/aclimdb/train/pos\10005_7.txt
./data/aclimdb/train/pos\10006_7.txt
./data/aclimdb/train/pos\10007_7.txt
./data/aclimdb/train/pos\10008_7.txt
./data/aclimdb/train/pos\10009_9.txt
./data/aclimdb/train/pos\1000_8.txt
./data/aclimdb/train/pos\10010_7.txt
./data/aclimdb/train/pos\10011_9.txt
./data/aclimdb/train/pos\10012_8.txt
./data/aclimdb/train/pos\10013_7.txt
./data/aclimdb/train/pos\10014_8.txt
./data/aclimdb/train/pos\10015_8.txt
./data/aclimdb/train/pos\10016_8.txt
./data/aclimdb/train/pos\10017_9.txt
./data/aclimdb/train/pos\10018_8.txt
./data/aclimdb/train/pos\10019_8.txt
./data/aclimdb/train/pos\1001_8.txt
./data/aclimdb/train/pos\10020_8.txt
./data/aclimdb/train/pos\10021_8.txt
./data/aclimdb/train/pos\10022_7.txt
./data/aclimdb/train/pos\10023_9.txt
./data

./data/aclimdb/train/pos\10364_10.txt
./data/aclimdb/train/pos\10365_8.txt
./data/aclimdb/train/pos\10366_10.txt
./data/aclimdb/train/pos\10367_8.txt
./data/aclimdb/train/pos\10368_7.txt
./data/aclimdb/train/pos\10369_8.txt
./data/aclimdb/train/pos\1036_9.txt
./data/aclimdb/train/pos\10370_9.txt
./data/aclimdb/train/pos\10371_8.txt
./data/aclimdb/train/pos\10372_7.txt
./data/aclimdb/train/pos\10373_7.txt
./data/aclimdb/train/pos\10374_8.txt
./data/aclimdb/train/pos\10375_10.txt
./data/aclimdb/train/pos\10376_7.txt
./data/aclimdb/train/pos\10377_9.txt
./data/aclimdb/train/pos\10378_8.txt
./data/aclimdb/train/pos\10379_10.txt
./data/aclimdb/train/pos\1037_8.txt
./data/aclimdb/train/pos\10380_10.txt
./data/aclimdb/train/pos\10381_10.txt
./data/aclimdb/train/pos\10382_10.txt
./data/aclimdb/train/pos\10383_10.txt
./data/aclimdb/train/pos\10384_10.txt
./data/aclimdb/train/pos\10385_10.txt
./data/aclimdb/train/pos\10386_8.txt
./data/aclimdb/train/pos\10387_7.txt
./data/aclimdb/train/pos\10388

./data/aclimdb/train/pos\10726_7.txt
./data/aclimdb/train/pos\10727_7.txt
./data/aclimdb/train/pos\10728_10.txt
./data/aclimdb/train/pos\10729_8.txt
./data/aclimdb/train/pos\1072_10.txt
./data/aclimdb/train/pos\10730_10.txt
./data/aclimdb/train/pos\10731_7.txt
./data/aclimdb/train/pos\10732_8.txt
./data/aclimdb/train/pos\10733_7.txt
./data/aclimdb/train/pos\10734_10.txt
./data/aclimdb/train/pos\10735_10.txt
./data/aclimdb/train/pos\10736_10.txt
./data/aclimdb/train/pos\10737_10.txt
./data/aclimdb/train/pos\10738_9.txt
./data/aclimdb/train/pos\10739_10.txt
./data/aclimdb/train/pos\1073_9.txt
./data/aclimdb/train/pos\10740_8.txt
./data/aclimdb/train/pos\10741_10.txt
./data/aclimdb/train/pos\10742_9.txt
./data/aclimdb/train/pos\10743_9.txt
./data/aclimdb/train/pos\10744_8.txt
./data/aclimdb/train/pos\10745_10.txt
./data/aclimdb/train/pos\10746_10.txt
./data/aclimdb/train/pos\10747_10.txt
./data/aclimdb/train/pos\10748_10.txt
./data/aclimdb/train/pos\10749_8.txt
./data/aclimdb/train/pos\10

./data/aclimdb/train/pos\11102_10.txt
./data/aclimdb/train/pos\11103_10.txt
./data/aclimdb/train/pos\11104_10.txt
./data/aclimdb/train/pos\11105_10.txt
./data/aclimdb/train/pos\11106_10.txt
./data/aclimdb/train/pos\11107_10.txt
./data/aclimdb/train/pos\11108_10.txt
./data/aclimdb/train/pos\11109_9.txt
./data/aclimdb/train/pos\1110_9.txt
./data/aclimdb/train/pos\11110_9.txt
./data/aclimdb/train/pos\11111_9.txt
./data/aclimdb/train/pos\11112_7.txt
./data/aclimdb/train/pos\11113_9.txt
./data/aclimdb/train/pos\11114_10.txt
./data/aclimdb/train/pos\11115_10.txt
./data/aclimdb/train/pos\11116_10.txt
./data/aclimdb/train/pos\11117_10.txt
./data/aclimdb/train/pos\11118_10.txt
./data/aclimdb/train/pos\11119_10.txt
./data/aclimdb/train/pos\1111_10.txt
./data/aclimdb/train/pos\11120_10.txt
./data/aclimdb/train/pos\11121_10.txt
./data/aclimdb/train/pos\11122_10.txt
./data/aclimdb/train/pos\11123_10.txt
./data/aclimdb/train/pos\11124_10.txt
./data/aclimdb/train/pos\11125_10.txt
./data/aclimdb/train

./data/aclimdb/train/pos\11434_7.txt
./data/aclimdb/train/pos\11435_10.txt
./data/aclimdb/train/pos\11436_9.txt
./data/aclimdb/train/pos\11437_9.txt
./data/aclimdb/train/pos\11438_10.txt
./data/aclimdb/train/pos\11439_8.txt
./data/aclimdb/train/pos\1143_7.txt
./data/aclimdb/train/pos\11440_10.txt
./data/aclimdb/train/pos\11441_10.txt
./data/aclimdb/train/pos\11442_10.txt
./data/aclimdb/train/pos\11443_10.txt
./data/aclimdb/train/pos\11444_10.txt
./data/aclimdb/train/pos\11445_7.txt
./data/aclimdb/train/pos\11446_10.txt
./data/aclimdb/train/pos\11447_7.txt
./data/aclimdb/train/pos\11448_10.txt
./data/aclimdb/train/pos\11449_10.txt
./data/aclimdb/train/pos\1144_8.txt
./data/aclimdb/train/pos\11450_10.txt
./data/aclimdb/train/pos\11451_8.txt
./data/aclimdb/train/pos\11452_8.txt
./data/aclimdb/train/pos\11453_9.txt
./data/aclimdb/train/pos\11454_10.txt
./data/aclimdb/train/pos\11455_8.txt
./data/aclimdb/train/pos\11456_10.txt
./data/aclimdb/train/pos\11457_10.txt
./data/aclimdb/train/pos\1

...

./data/aclimdb/train/pos\8779_8.txt
./data/aclimdb/train/pos\877_8.txt
./data/aclimdb/train/pos\8780_10.txt
./data/aclimdb/train/pos\8781_9.txt
./data/aclimdb/train/pos\8782_9.txt
./data/aclimdb/train/pos\8783_9.txt
./data/aclimdb/train/pos\8784_9.txt
./data/aclimdb/train/pos\8785_10.txt
./data/aclimdb/train/pos\8786_8.txt
./data/aclimdb/train/pos\8787_8.txt
./data/aclimdb/train/pos\8788_10.txt
./data/aclimdb/train/pos\8789_10.txt
./data/aclimdb/train/pos\878_7.txt
./data/aclimdb/train/pos\8790_8.txt
./data/aclimdb/train/pos\8791_10.txt
./data/aclimdb/train/pos\8792_8.txt
./data/aclimdb/train/pos\8793_10.txt
./data/aclimdb/train/pos\8794_9.txt
./data/aclimdb/train/pos\8795_10.txt
./data/aclimdb/train/pos\8796_8.txt
./data/aclimdb/train/pos\8797_8.txt
./data/aclimdb/train/pos\8798_8.txt
./data/aclimdb/train/pos\8799_8.txt
./data/aclimdb/train/pos\879_8.txt
./data/aclimdb/train/pos\87_10.txt
./data/aclimdb/train/pos\8800_10.txt
./data/aclimdb/train/pos\8801_9.txt
./data/aclimdb/train/pos

./data/aclimdb/train/pos\9133_9.txt
./data/aclimdb/train/pos\9134_7.txt
./data/aclimdb/train/pos\9135_9.txt
./data/aclimdb/train/pos\9136_8.txt
./data/aclimdb/train/pos\9137_10.txt
./data/aclimdb/train/pos\9138_10.txt
./data/aclimdb/train/pos\9139_8.txt
./data/aclimdb/train/pos\913_10.txt
./data/aclimdb/train/pos\9140_10.txt
./data/aclimdb/train/pos\9141_8.txt
./data/aclimdb/train/pos\9142_10.txt
./data/aclimdb/train/pos\9143_10.txt
./data/aclimdb/train/pos\9144_7.txt
./data/aclimdb/train/pos\9145_10.txt
./data/aclimdb/train/pos\9146_9.txt
./data/aclimdb/train/pos\9147_10.txt
./data/aclimdb/train/pos\9148_7.txt
./data/aclimdb/train/pos\9149_9.txt
./data/aclimdb/train/pos\914_8.txt
./data/aclimdb/train/pos\9150_8.txt
./data/aclimdb/train/pos\9151_10.txt
./data/aclimdb/train/pos\9152_8.txt
./data/aclimdb/train/pos\9153_8.txt
./data/aclimdb/train/pos\9154_7.txt
./data/aclimdb/train/pos\9155_10.txt
./data/aclimdb/train/pos\9156_10.txt
./data/aclimdb/train/pos\9157_10.txt
./data/aclimdb/tra

./data/aclimdb/train/pos\9464_8.txt
./data/aclimdb/train/pos\9465_7.txt
./data/aclimdb/train/pos\9466_7.txt
./data/aclimdb/train/pos\9467_10.txt
./data/aclimdb/train/pos\9468_7.txt
./data/aclimdb/train/pos\9469_10.txt
./data/aclimdb/train/pos\946_8.txt
./data/aclimdb/train/pos\9470_8.txt
./data/aclimdb/train/pos\9471_7.txt
./data/aclimdb/train/pos\9472_8.txt
./data/aclimdb/train/pos\9473_10.txt
./data/aclimdb/train/pos\9474_10.txt
./data/aclimdb/train/pos\9475_9.txt
./data/aclimdb/train/pos\9476_9.txt
./data/aclimdb/train/pos\9477_9.txt
./data/aclimdb/train/pos\9478_8.txt
./data/aclimdb/train/pos\9479_10.txt
./data/aclimdb/train/pos\947_10.txt
./data/aclimdb/train/pos\9480_10.txt
./data/aclimdb/train/pos\9481_9.txt
./data/aclimdb/train/pos\9482_7.txt
./data/aclimdb/train/pos\9483_10.txt
./data/aclimdb/train/pos\9484_10.txt
./data/aclimdb/train/pos\9485_10.txt
./data/aclimdb/train/pos\9486_10.txt
./data/aclimdb/train/pos\9487_8.txt
./data/aclimdb/train/pos\9488_10.txt
./data/aclimdb/tra

./data/aclimdb/train/pos\9783_9.txt
./data/aclimdb/train/pos\9784_8.txt
./data/aclimdb/train/pos\9785_7.txt
./data/aclimdb/train/pos\9786_8.txt
./data/aclimdb/train/pos\9787_7.txt
./data/aclimdb/train/pos\9788_9.txt
./data/aclimdb/train/pos\9789_10.txt
./data/aclimdb/train/pos\978_9.txt
./data/aclimdb/train/pos\9790_9.txt
./data/aclimdb/train/pos\9791_9.txt
./data/aclimdb/train/pos\9792_8.txt
./data/aclimdb/train/pos\9793_7.txt
./data/aclimdb/train/pos\9794_7.txt
./data/aclimdb/train/pos\9795_7.txt
./data/aclimdb/train/pos\9796_9.txt
./data/aclimdb/train/pos\9797_7.txt
./data/aclimdb/train/pos\9798_7.txt
./data/aclimdb/train/pos\9799_7.txt
./data/aclimdb/train/pos\979_8.txt
./data/aclimdb/train/pos\97_9.txt
./data/aclimdb/train/pos\9800_9.txt
./data/aclimdb/train/pos\9801_10.txt
./data/aclimdb/train/pos\9802_10.txt
./data/aclimdb/train/pos\9803_7.txt
./data/aclimdb/train/pos\9804_10.txt
./data/aclimdb/train/pos\9805_10.txt
./data/aclimdb/train/pos\9806_8.txt
./data/aclimdb/train/pos\98

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 194: character maps to <undefined>

<img src="./images/pooling.png"/>

In [26]:
dataset

[]